In [20]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import undetected_chromedriver as uc

import sys
import time
from bs4 import BeautifulSoup
import pandas as pd
import re

In [21]:
def expend_transparency_section(data,wd):
    transparency = data.find_elements(By.CSS_SELECTOR,'div.x1iyjqo2 > div[role="heading"]')
    t =0
    while not transparency and t < 10:  # try to find the transparency section, retry up to 10 times
        data = wd.find_element(By.CSS_SELECTOR,'body > [data-visualcompletion=ignore]')
        transparency = data.find_elements(By.CSS_SELECTOR,'div.x1iyjqo2 > div[role="button"]')
        t += 1
        time.sleep(1)
    if t >= 10:
        print("Transparency section not found after 10 attempts.")
        return False
    if transparency:
        for i in transparency:
            try:
                if i.text == 'Transparency by location':
                    transparency = i
                    time.sleep(1)
                    ActionChains(wd).move_to_element(transparency).click().perform()  # expand the transparency section
                    time.sleep(1)
                    return True
            except Exception as e:
                continue
        else:
            data = wd.find_element(By.CSS_SELECTOR,'body > [data-visualcompletion=ignore]')
            expend_transparency_section(data,wd)  # if not found, try to find element again

In [22]:
def extract_ad_details(wd,company_name):
    data = wd.find_element(By.CSS_SELECTOR,'body > [data-visualcompletion=ignore]')
    html_content = data.get_attribute('innerHTML')
    ad_data = {}
    
    # Extract Library ID
    library_id = re.search(r'Library ID:\s*(\d+)', html_content)
    t = 0
    while not library_id and t<5: # If not found, try to find element again
        data = wd.find_element(By.CSS_SELECTOR,'body > [data-visualcompletion=ignore]')
        html_content = data.get_attribute('innerHTML')
        library_id = re.search(r'Library ID:\s*(\d+)', html_content)
        t += 1
        time.sleep(1)  # wait for a second before trying again
    if library_id:
        library_id = library_id.groups()[0]
        library_id = library_id.replace('Library ID:', '').strip()
    else:
        library_id = None

    ad_data['library_id'] = library_id
    
    ad_data['company_name'] = company_name

    # Extract Ad Duration (dates)
    date_pattern = r'(\d{1,2}\s+\w+\s+\d{4})\s*-\s*(\d{1,2}\s+\w+\s+\d{4})'
    date_match = re.search(date_pattern, html_content)
    if date_match:
        ad_data['start_date'] = date_match.group(1)
        ad_data['end_date'] = date_match.group(2)
        ad_data['duration'] = f"{date_match.group(1)} - {date_match.group(2)}"
    else:
        ad_data['start_date'] = None
        ad_data['end_date'] = None
        ad_data['duration'] = None
    
    # Extract platfotm
    platforms = []
    platform_html = data.find_elements(By.CSS_SELECTOR,'div.x3nfvp2.x1e56ztr > div.x6s0dn4.x78zum5.x1q0g3np.x2lwn1j.xeuugli[style]')
    if platform_html:
        platform_html = platform_html[0]
        platform_html = platform_html.get_attribute('innerHTML')
        fb_icon_url = 'https://static.xx.fbcdn.net/rsrc.php/v4/yW/r/TP7nCDju1B-.png&quot'
        insta_icon_url = 'https://static.xx.fbcdn.net/rsrc.php/v4/ye/r/HhjbQ9EnZ_3.png&quot'
        audience_network_icon_url = 'https://static.xx.fbcdn.net/rsrc.php/v4/y3/r/r35dp7ubbrO.png&quot'
        msger_icon_url = 'https://static.xx.fbcdn.net/rsrc.php/v4/y3/r/r35dp7ubbrO.png&quot'
        threads_icon_url = 'https://static.xx.fbcdn.net/rsrc.php/v4/yW/r/TP7nCDju1B-.png&quot'
        if fb_icon_url in platform_html:
            platforms.append('Facebook')
        if insta_icon_url in platform_html:
            platforms.append('Instagram')
        if audience_network_icon_url in platform_html:
            platforms.append('Audience Network')
        if msger_icon_url in platform_html:
            platforms.append('Messenger')
        if threads_icon_url in platform_html:
            platforms.append('Threads')
        platforms = ', '.join(platforms)  #join platforms with commas
        ad_data['platforms'] = platforms
    else:
        ad_data['platforms'] = None

    # Extract Ad Text
    ads_text = data.find_element(By.CSS_SELECTOR, '._4ik4._4ik5 > div[style] >span')
    ads_text = ads_text.text
    
    ad_data['ad_text'] = ads_text
    
    # Extract Reach (viewer count)
    reach = data.find_elements(By.CSS_SELECTOR, '.x1cy8zhl.x78zum5.xdt5ytf.x2lwn1j.xeuugli.x3pnbk8.xyamay9.xv54qhq.x1l90r2v.xf7dkkf.x1kmqopl.x1obq294.x5a5i1n.xde0f50.x15x8krk '
)
    for element in reach:
        titles = element.find_elements(By.CSS_SELECTOR, '.xuk3077.x78zum5.x2lwn1j.xeuugli.xfex06f')
        for title in titles:
            if title.text == 'Reach':
                reach = element
                break
    reach = reach.find_element(By.CSS_SELECTOR, '.x8t9es0.x10d9sdx.xo1l8bm.xrohxju.x108nfp6.xq9mrsl.x1h4wwuj.xeuugli')
    reach = reach.text
    ad_data['reach'] = reach
    
    # Determine if it's video or image
    video = data.find_elements(By.CSS_SELECTOR, 'video')
    img = data.find_elements(By.CSS_SELECTOR, 'img')
    if video and img:
        ad_data['media_type'] = 'video&image'
    elif video:
        ad_data['media_type'] = 'video'
    elif img:
        ad_data['media_type'] = 'image'

    # extract images url if it's an image ad
    img_urls = []
    if img:
        for image in img:
            image_url = image.get_attribute('src')
            if image_url:
                img_urls.append(image_url)
        image_urls = ','.join(img_urls)
        ad_data['image_urls'] = image_urls
    else:
        ad_data['image_url'] = None

    return ad_data

In [23]:
def progress_bar(i,total):
    percent = (i / total) * 100
    bar = '#' * int(percent // 2) + '-' * (50 - int(percent // 2))
    sys.stdout.write(f"\r[{bar}] {percent:.2f}%")
    sys.stdout.flush()

In [ ]:
def ads_scraper(company_name,ads, active_status='inactive',n_scroll=5):
    active_status = 'inactive' # viewers reached only available for inactive ads

    op = Options()
    op.add_argument("--headless")  # Run in headless mode
    op.add_argument("--no-sandbox")
    op.add_argument('--disable-gpu')
    op.add_argument('--incognito')
    op.add_argument("--disable-blink-features=AutomationControlled")
    op.add_argument('load-extension=C:\\Users\\Ten Yee Chern\\AppData\\Local\\Google\\Chrome\\User Data\\Default\\Extensions\\ailoabdmgclmfmhdagmlohpjlbpffblp')

    op.add_experimental_option('useAutomationExtension', False)
    op.add_experimental_option('excludeSwitches', ['enable-automation'])


    wd = webdriver.Chrome(options=op)

    wd.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
        'source': """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        });
        """})

    wd.implicitly_wait(10)  # Set an implicit wait of 10 seconds

    wd.get(f'https://www.facebook.com/ads/library/?active_status={active_status}&ad_type=all&country=ALL&is_targeted_country=false&media_type=all&search_type=page&view_all_page_id={companies_id[company_name]}')

    for _ in range(n_scroll):  # Default is scroll down 5 times
        wd.execute_script("window.scrollTo(0, document.body.scrollHeight);") # Scroll to the bottom of the page
        time.sleep(5)  # Wait for the page to load
    
    elements = wd.find_elements(By.CSS_SELECTOR, '.x1plvlek.xryxfnj.x1gzqxud.x178xt8z.x1lun4ml.xso031l.xpilrb4.xb9moi8.xe76qn7.x21b0me.x142aazg.x1i5p2am.x1whfx0g.xr2y4jy.x1ihp6rs.x1kmqopl.x13fuv20.x18b5jzi.x1q0q8m5.x1t7ytsu.x9f619   .x8t9es0.x1fvot60.xxio538.x1heor9g.xuxw1ft.x6ikm8r.x10wlt62.xlyipyv.x1h4wwuj.x1pd3egz.xeuugli')

    details = []
    for element in elements:
        if element.text == 'See ad details':
            details.append(element)
    print(len(details), "ads found for", company_name, '\nExtracting ad details...')
    
    for i,detail in enumerate(details):
        progress_bar(i, len(details))   # Print progress bar

        wd.execute_script('arguments[0].click();', detail)  # Click on the ad details link
        time.sleep(1)  # Wait for the ad details to load
        data = wd.find_element(By.CSS_SELECTOR,'body > [data-visualcompletion=ignore]')
        
        transparency_click = expend_transparency_section(data,wd)
        if transparency_click:
            ad_data = extract_ad_details(wd, company_name)
            ads.append(ad_data)

        # close the ad details popup
        details_close = wd.find_element(By.CSS_SELECTOR, '.x7a106z.x78zum5.x2lah0s.x9otpla.x1wsgfga.x1w5wx5t > span.x13fj5qh > .x1i10hfl.xjqpnuy.xc5r6h4.xqeqjp1.x1phubyo.x972fbf.x10w94by.x1qhh985.x14e42zd.x9f619.x1ypdohk.xe8uvvx.xdj266r.x14z9mp.xat24cr.x1lziwak.x2lwn1j.xeuugli.x16tdsg8.xggy1nq.x1ja2u2z.x1t137rt.x6s0dn4.x1ejq31n.x18oe1m7.x1sy0etr.xstzfhl.x3nfvp2.xdl72j9.x1q0g3np.x2lah0s.x193iq5w.x1n2onr6.x1hl2dhg.x87ps6o.xxymvpz.xlh3980.xvmahel.x1lku1pv.x1i5p2am.x1whfx0g.xr2y4jy.x1ihp6rs.xo1l8bm.x108nfp6.x1v911su.x1y1aw1k.xwib8y2.x1g0dm76.xpdmqnj')
        ActionChains(wd).move_to_element(details_close).click().perform()

        i+= 1
        progress_bar(i, len(details))  # Update progress bar after one ad is processed
    
    wd.quit()  # Close the browser after scraping is done
        

In [28]:
ads = []
companies = ['AG1 by Athletic Greens', 'IM8 Health', 'IM8 Health Global','Loop']
companies_id = {
    'AG1 by Athletic Greens': 183869772601,
    'IM8 Health': 345914995276546,
    'IM8 Health Global': 683327568197335,
    'Loop': 517850318391712,
}
for _ in companies:
    ads.append([])
#     ads_scraper(company, ads=ads)
#     print(f"\nCompleted scraping for {company}, {len(ads)} ads collected.")


In [ ]:
success = False
while not success:
    ads[0] = []
    try:
        ads_scraper(companies[0], ads=ads[0],n_scroll=5)  # increase n_scroll for more ads
        success = True
    except Exception as e:
        print(f"Error scraping {companies[0]}: {e}")
        success = False

87 ads found for AG1 by Athletic Greens 
Extracting ad details...
[##################################################] 100.00%

In [ ]:
success = False
while not success:
    ads[1] =[]
    try:
        ads_scraper(companies[1], ads=ads[1],n_scroll=5)  # increase n_scroll for more ads
        success = True
    except Exception as e:
        print(f"Error scraping {companies[1]}: {e}")
        success = False

171 ads found for IM8 Health 
Extracting ad details...
[##################################################] 100.00%

In [ ]:
success = False
while not success:
    ads[2] = []
    try:
        ads_scraper(companies[2], ads=ads[2],n_scroll=5)  # increase n_scroll for more ads
        success = True
    except Exception as e:
        print(f"Error scraping {companies[2]}: {e}")
        success = False

145 ads found for IM8 Health Global 
Extracting ad details...
[##################################################] 100.00%

In [ ]:
success = False
while not success:
    ads[3] =[]
    try:
        ads_scraper(companies[3], ads=ads[3],n_scroll=5)  # increase n_scroll for more ads
        success = True
    except Exception as e:
        print(f"Error scraping {companies[3]}: {e}")
        success = False

Error scraping Loop: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=138.0.7204.169)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ef10e935+77845]
	GetHandleVerifier [0x0x7ff6ef10e990+77936]
	(No symbol) [0x0x7ff6eeec9cda]
	(No symbol) [0x0x7ff6eeec6a93]
	(No symbol) [0x0x7ff6eeeb7899]
	(No symbol) [0x0x7ff6eeeb9651]
	(No symbol) [0x0x7ff6eeeb7bb6]
	(No symbol) [0x0x7ff6eeeb7616]
	(No symbol) [0x0x7ff6eeeb72da]
	(No symbol) [0x0x7ff6eeeb4eab]
	(No symbol) [0x0x7ff6eeeb572c]
	(No symbol) [0x0x7ff6eeecdc8a]
	(No symbol) [0x0x7ff6eef718be]
	(No symbol) [0x0x7ff6eef488ca]
	(No symbol) [0x0x7ff6eef70b07]
	(No symbol) [0x0x7ff6eef486a3]
	(No symbol) [0x0x7ff6eef11791]
	(No symbol) [0x0x7ff6eef12523]
	GetHandleVerifier [0x0x7ff6ef3e684d+3059501]
	GetHandleVerifier [0x0x7ff6ef3e0c0d+3035885]
	GetHandleVerifier [0x0x7ff6ef400400+3164896]
	GetHandleVerifier [0x0x7ff6ef128c3e+185118]
	GetHandleVerifier [0x0x7ff6ef13054f+216111]
	GetHandleVerifier [0x0x7ff6ef1172e4+113092]
	Ge

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
Error sending stats to Plausible: error sending request for url (https://plausible.io/api/event)
There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error scraping Loop: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=138.0.7204.169)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ef10e935+77845]
	GetHandleVerifier [0x0x7ff6ef10e990+77936]
	(No symbol) [0x0x7ff6eeec9cda]
	(No symbol) [0x0x7ff6eeec6a93]
	(No symbol) [0x0x7ff6eeeb7899]
	(No symbol) [0x0x7ff6eeeb9651]
	(No symbol) [0x0x7ff6eeeb7bb6]
	(No symbol) [0x0x7ff6eeeb7616]
	(No symbol) [0x0x7ff6eeeb72da]
	(No symbol) [0x0x7ff6eeeb4eab]
	(No symbol) [0x0x7ff6eeeb572c]
	(No symbol) [0x0x7ff6eeecdc8a]
	(No symbol) [0x0x7ff6eef718be]
	(No symbol) [0x0x7ff6eef488ca]
	(No symbol) [0x0x7ff6eef70b07]
	(No symbol) [0x0x7ff6eef486a3]
	(No symbol) [0x0x7ff6eef11791]
	(No symbol) [0x0x7ff6eef12523]
	GetHandleVerifier [0x0x7ff6ef3e684d+3059501]
	GetHandleVerifier [0x0x7ff6ef3e0c0d+3035885]
	GetHandleVerifier [0x0x7ff6ef400400+3164896]
	GetHandleVerifier [0x0x7ff6ef128c3e+185118]
	GetHandleVerifier [0x0x7ff6ef13054f+216111]
	GetHandleVerifier [0x0x7ff6ef1172e4+113092]
	Ge

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache
There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error scraping Loop: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=138.0.7204.169)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ef10e935+77845]
	GetHandleVerifier [0x0x7ff6ef10e990+77936]
	(No symbol) [0x0x7ff6eeec9cda]
	(No symbol) [0x0x7ff6eeec6a93]
	(No symbol) [0x0x7ff6eeeb7899]
	(No symbol) [0x0x7ff6eeeb9651]
	(No symbol) [0x0x7ff6eeeb7bb6]
	(No symbol) [0x0x7ff6eeeb7616]
	(No symbol) [0x0x7ff6eeeb72da]
	(No symbol) [0x0x7ff6eeeb4eab]
	(No symbol) [0x0x7ff6eeeb572c]
	(No symbol) [0x0x7ff6eeecdc8a]
	(No symbol) [0x0x7ff6eef718be]
	(No symbol) [0x0x7ff6eef488ca]
	(No symbol) [0x0x7ff6eef70b07]
	(No symbol) [0x0x7ff6eef486a3]
	(No symbol) [0x0x7ff6eef11791]
	(No symbol) [0x0x7ff6eef12523]
	GetHandleVerifier [0x0x7ff6ef3e684d+3059501]
	GetHandleVerifier [0x0x7ff6ef3e0c0d+3035885]
	GetHandleVerifier [0x0x7ff6ef400400+3164896]
	GetHandleVerifier [0x0x7ff6ef128c3e+185118]
	GetHandleVerifier [0x0x7ff6ef13054f+216111]
	GetHandleVerifier [0x0x7ff6ef1172e4+113092]
	Ge

...

Error scraping Loop: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=138.0.7204.169)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ef10e935+77845]
	GetHandleVerifier [0x0x7ff6ef10e990+77936]
	(No symbol) [0x0x7ff6eeec9cda]
	(No symbol) [0x0x7ff6eeec6a93]
	(No symbol) [0x0x7ff6eeeb7899]
	(No symbol) [0x0x7ff6eeeb9651]
	(No symbol) [0x0x7ff6eeeb7bb6]
	(No symbol) [0x0x7ff6eeeb7616]
	(No symbol) [0x0x7ff6eeeb72da]
	(No symbol) [0x0x7ff6eeeb4eab]
	(No symbol) [0x0x7ff6eeeb572c]
	(No symbol) [0x0x7ff6eeecdc8a]
	(No symbol) [0x0x7ff6eef718be]
	(No symbol) [0x0x7ff6eef488ca]
	(No symbol) [0x0x7ff6eef70b07]
	(No symbol) [0x0x7ff6eef486a3]
	(No symbol) [0x0x7ff6eef11791]
	(No symbol) [0x0x7ff6eef12523]
	GetHandleVerifier [0x0x7ff6ef3e684d+3059501]
	GetHandleVerifier [0x0x7ff6ef3e0c0d+3035885]
	GetHandleVerifier [0x0x7ff6ef400400+3164896]
	GetHandleVerifier [0x0x7ff6ef128c3e+185118]
	GetHandleVerifier [0x0x7ff6ef13054f+216111]
	GetHandleVerifier [0x0x7ff6ef1172e4+113092]
	Ge

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error scraping Loop: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=138.0.7204.169)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ef10e935+77845]
	GetHandleVerifier [0x0x7ff6ef10e990+77936]
	(No symbol) [0x0x7ff6eeec9cda]
	(No symbol) [0x0x7ff6eeec6a93]
	(No symbol) [0x0x7ff6eeeb7899]
	(No symbol) [0x0x7ff6eeeb9651]
	(No symbol) [0x0x7ff6eeeb7bb6]
	(No symbol) [0x0x7ff6eeeb7616]
	(No symbol) [0x0x7ff6eeeb72da]
	(No symbol) [0x0x7ff6eeeb4eab]
	(No symbol) [0x0x7ff6eeeb572c]
	(No symbol) [0x0x7ff6eeecdc8a]
	(No symbol) [0x0x7ff6eef718be]
	(No symbol) [0x0x7ff6eef488ca]
	(No symbol) [0x0x7ff6eef70b07]
	(No symbol) [0x0x7ff6eef486a3]
	(No symbol) [0x0x7ff6eef11791]
	(No symbol) [0x0x7ff6eef12523]
	GetHandleVerifier [0x0x7ff6ef3e684d+3059501]
	GetHandleVerifier [0x0x7ff6ef3e0c0d+3035885]
	GetHandleVerifier [0x0x7ff6ef400400+3164896]
	GetHandleVerifier [0x0x7ff6ef128c3e+185118]
	GetHandleVerifier [0x0x7ff6ef13054f+216111]
	GetHandleVerifier [0x0x7ff6ef1172e4+113092]
	Ge

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


Error scraping Loop: Message: unknown error: net::ERR_NAME_NOT_RESOLVED
  (Session info: chrome=138.0.7204.169)
Stacktrace:
	GetHandleVerifier [0x0x7ff6ef10e935+77845]
	GetHandleVerifier [0x0x7ff6ef10e990+77936]
	(No symbol) [0x0x7ff6eeec9cda]
	(No symbol) [0x0x7ff6eeec6a93]
	(No symbol) [0x0x7ff6eeeb7899]
	(No symbol) [0x0x7ff6eeeb9651]
	(No symbol) [0x0x7ff6eeeb7bb6]
	(No symbol) [0x0x7ff6eeeb7616]
	(No symbol) [0x0x7ff6eeeb72da]
	(No symbol) [0x0x7ff6eeeb4eab]
	(No symbol) [0x0x7ff6eeeb572c]
	(No symbol) [0x0x7ff6eeecdc8a]
	(No symbol) [0x0x7ff6eef718be]
	(No symbol) [0x0x7ff6eef488ca]
	(No symbol) [0x0x7ff6eef70b07]
	(No symbol) [0x0x7ff6eef486a3]
	(No symbol) [0x0x7ff6eef11791]
	(No symbol) [0x0x7ff6eef12523]
	GetHandleVerifier [0x0x7ff6ef3e684d+3059501]
	GetHandleVerifier [0x0x7ff6ef3e0c0d+3035885]
	GetHandleVerifier [0x0x7ff6ef400400+3164896]
	GetHandleVerifier [0x0x7ff6ef128c3e+185118]
	GetHandleVerifier [0x0x7ff6ef13054f+216111]
	GetHandleVerifier [0x0x7ff6ef1172e4+113092]
	Ge

In [38]:
ads_data = []
for ad in ads:
    ads_data.extend(ad)

In [39]:
df = pd.DataFrame(ads_data,columns=ads_data[0].keys())
df

,library_id,company_name,start_date,end_date,duration,platforms,ad_text,reach,media_type,image_urls
0,4021324708082706,AG1 by Athletic Greens,24 Jul 2025,25 Jul 2025,24 Jul 2025 - 25 Jul 2025,"Facebook, Audience Network, Messenger, Threads","When performance matters, recovery matters mor...","1,865",video&image,https://scontent-sin2-3.xx.fbcdn.net/v/t39.354...
1,4187438514821208,AG1 by Athletic Greens,24 Jul 2025,27 Jul 2025,24 Jul 2025 - 27 Jul 2025,"Facebook, Audience Network, Messenger, Threads",See the upgrade for yourself! 👇\n\nHave you se...,"13,907",video&image,https://scontent-sin11-1.xx.fbcdn.net/v/t39.35...
2,2487179494979343,AG1 by Athletic Greens,23 Jul 2025,25 Jul 2025,23 Jul 2025 - 25 Jul 2025,"Facebook, Audience Network, Messenger, Threads","When performance is everything, the small thin...","26,002",video&image,https://scontent-sin2-3.xx.fbcdn.net/v/t39.354...
3,1526768818697884,AG1 by Athletic Greens,22 Jul 2025,24 Jul 2025,22 Jul 2025 - 24 Jul 2025,"Facebook, Audience Network, Messenger, Threads",Always on? Always busy? AG1 is a one-scoop sol...,"19,108",video&image,https://scontent-sin11-2.xx.fbcdn.net/v/t39.35...
4,721730560833506,AG1 by Athletic Greens,22 Jul 2025,23 Jul 2025,22 Jul 2025 - 23 Jul 2025,"Facebook, Audience Network, Messenger, Threads",Poolside? Workout? Packed summer plans?\nAG1 h...,"37,676",image,https://scontent-sin6-1.xx.fbcdn.net/v/t39.354...
...,...,...,...,...,...,...,...,...,...,...
434,722320090685682,Loop,24 Jul 2025,25 Jul 2025,24 Jul 2025 - 25 Jul 2025,"Facebook, Audience Network, Messenger, Threads",What's even better than your favorite Loop ear...,"20,013",video&image,https://scontent-sin6-3.xx.fbcdn.net/v/t39.354...
435,4054422141541920,Loop,24 Jul 2025,24 Jul 2025,24 Jul 2025 - 24 Jul 2025,"Facebook, Audience Network, Messenger, Threads",Did you know that hearing damage is often irre...,"1,721",video&image,https://scontent-sin6-3.xx.fbcdn.net/v/t39.354...
436,724442850412886,Loop,24 Jul 2025,24 Jul 2025,24 Jul 2025 - 24 Jul 2025,"Facebook, Audience Network, Messenger, Threads",Want earplugs that can take you anywhere? 🙋‍♂️...,"5,272",video&image,https://scontent-sin11-2.xx.fbcdn.net/v/t39.35...
437,1513308633373664,Loop,24 Jul 2025,27 Jul 2025,24 Jul 2025 - 27 Jul 2025,"Facebook, Audience Network, Messenger, Threads",Don’t forget! 🚀 Tomorrowland x Loop earplugs a...,"10,184",video&image,https://scontent-sin6-3.xx.fbcdn.net/v/t39.354...


In [40]:
df['start_date'] = pd.to_datetime(df['start_date'])
df['end_date'] = pd.to_datetime(df['end_date'])
df['duration'] = df['end_date'] - df['start_date']

In [41]:
df['text_length'] = df['ad_text'].apply(lambda x: len(x.split(' ')))  # Calculate words

In [46]:
df['reach'] = df['reach'].apply(lambda x : x.replace(',','')).astype(int)  # Convert reach to integer
df.sort_values(by='reach', ascending=False)

,library_id,company_name,start_date,end_date,duration,platforms,ad_text,reach,media_type,image_urls,text_length
48,1766087127365730,AG1 by Athletic Greens,2025-07-02,2025-07-11,9 days,"Facebook, Audience Network, Messenger, Threads","Bereit, jeden Tag alles zu geben? AG1 unterstü...",157422,video&image,https://scontent-sin6-3.xx.fbcdn.net/v/t39.354...,87
47,1483616936343154,AG1 by Athletic Greens,2025-07-03,2025-07-14,11 days,"Facebook, Audience Network, Messenger, Threads",Ready to give it your all every day? AG1 suppo...,134029,video&image,https://scontent-sin2-3.xx.fbcdn.net/v/t39.354...,94
41,1036675808657199,AG1 by Athletic Greens,2025-07-04,2025-07-07,3 days,"Facebook, Audience Network, Messenger, Threads",Ready to give it your all every day? AG1 suppo...,104232,video&image,https://scontent-sin2-1.xx.fbcdn.net/v/t39.354...,94
54,1823715901527733,AG1 by Athletic Greens,2025-07-01,2025-07-21,20 days,"Facebook, Audience Network, Messenger, Threads",Klaar om elke dag alles te kunnen geven? Laat ...,103232,image,https://scontent-sin11-1.xx.fbcdn.net/v/t39.35...,79
11,1301852304792381,AG1 by Athletic Greens,2025-07-21,2025-07-23,2 days,"Facebook, Audience Network, Messenger, Threads","Bereit, jeden Tag alles zu geben? AG1 unterstü...",95338,image,https://scontent-sin2-1.xx.fbcdn.net/v/t39.354...,87
...,...,...,...,...,...,...,...,...,...,...,...
201,1046112611040887,IM8 Health,2025-07-02,2025-07-09,7 days,"Facebook, Audience Network, Messenger, Threads","When it comes to your health, doses matter. So...",1,image,https://scontent-sin11-1.xx.fbcdn.net/v/t39.35...,119
98,1082421500504489,IM8 Health,2025-07-24,2025-07-24,0 days,"Facebook, Audience Network, Messenger, Threads",Here’s what people are saying—and why IM8 is n...,1,image,https://scontent-sin6-2.xx.fbcdn.net/v/t39.354...,98
93,1771535363437344,IM8 Health,2025-07-24,2025-07-24,0 days,"Facebook, Audience Network, Messenger, Threads",Co-Founding Partner David Beckham starts his d...,1,image,https://scontent-sin11-1.xx.fbcdn.net/v/t39.35...,42
102,1001776308526794,IM8 Health,2025-07-18,2025-07-26,8 days,"Facebook, Audience Network, Messenger, Threads",Co-Founding Partner David Beckham starts his d...,1,video&image,https://scontent-sin11-1.xx.fbcdn.net/v/t39.35...,42


In [47]:
df.to_csv('ads_data.csv', index=False)  # Save the DataFrame to a CSV file
print("Ads data saved to 'ads_data.csv'")  # Confirmation message

Ads data saved to 'ads_data.csv'


In [53]:
from IPython.display import Image, display
import requests

# show url images
reach_images = df[['reach', 'image_urls']].copy()
reach_images = reach_images.sort_values(by='reach', ascending=False)

for i in range(10): #show top 10 ads by reach
    urls = reach_images.iloc[i]['image_urls']
    urls = urls.split(',')
    for n,url in enumerate(urls):
        display(Image(url=url))
        # save the image to a file, path = f"content/ad_{i+1}.png"
        response = requests.get(url)
        with open(f"content/ad_{i}{n}.png", 'wb') as file:
            file.write(response.content)        